<a href="https://colab.research.google.com/github/yablockoo/GB/blob/PyTorch/HW_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feed-forward сети

Итак, давайте потренируемся тренировать нейронные сети прямого распространения 
При этом попробуем создать свою функцию активации на одном из слоев

In [ ]:
# Сделаем необходимые импорты
import torch
import numpy as np
from torch import optim
from torch import nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
# Загрузим датасет CIFAR-10, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)


image, label = train_dataset[0]
print (image.size())
print (label)

Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [ ]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False)

Files already downloaded and verified


In [ ]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации


class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 8 * hidden_dim)
        self.fc2 = nn.Linear(8 * hidden_dim, 16 * hidden_dim)
        self.fc3 = nn.Linear(16 * hidden_dim, 32 * hidden_dim)
        # self.fc4 = nn.Linear(32 * hidden_dim, 64 * hidden_dim)
        # self.fc5 = nn.Linear(64 * hidden_dim, 128 * hidden_dim) 
        # self.fc6 = nn.Linear(128 * hidden_dim, 256 * hidden_dim) 
        # self.fc7 = nn.Linear(256 * hidden_dim, 512 * hidden_dim)  
        self.fc_preout = nn.Linear(32 * hidden_dim, hidden_dim)   
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.05)
        x = self.fc2(x)
        x = F.leaky_relu(x, 0.05)
        x = self.fc3(x)
        x = F.leaky_relu(x, 0.05)
        # x = self.fc4(x)
        # x = F.leaky_relu(x, 0.05)
        # x = self.fc5(x)
        # x = F.leaky_relu(x, 0.05)
        # x = self.fc6(x)
        # x = F.leaky_relu(x, 0.05)
        # x = self.fc7(x)
        # x = F.leaky_relu(x, 0.05)
        x = self.fc_preout(x)
        x = F.leaky_relu(x, 0.05)
        x = self.fc_out(x)
        #x = F.softmax(x)
        return x


net = Net(3072, 100, 10)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Запустить обучение
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.001
[1,   301] loss: 0.345
[1,   601] loss: 0.343


 10%|█         | 1/10 [01:10<10:35, 70.63s/it]

[2,     1] loss: 0.001
[2,   301] loss: 0.321
[2,   601] loss: 0.310


 20%|██        | 2/10 [02:17<09:08, 68.62s/it]

...

[9,     1] loss: 0.001
[9,   301] loss: 0.244
[9,   601] loss: 0.240


 90%|█████████ | 9/10 [10:11<01:07, 67.52s/it]

[10,     1] loss: 0.001
[10,   301] loss: 0.237
[10,   601] loss: 0.236


100%|██████████| 10/10 [11:20<00:00, 68.03s/it]

Training is finished!
